In [1]:
%cd ../../src

e:\00_CODE\03_Master_Thesis\rdf-literal-preprocessing\src


In [2]:
import fire
import sys
import tqdm
import pandas as pd
import numpy as np
import kgbench as kg
from collections import Counter
import matplotlib.pyplot as pl
import random as rd
from operator import itemgetter
import torch
import torch.nn.functional as F
from utils import RDF_NUMBER_TYPES, get_relevant_relations, add_triple, get_p_types, ALL_LITERALS
from kgbench.load import Data
from typing import List, Sequence, Tuple

from kgbench import load, tic, toc, d
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
from utils import URI_PREFIX
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
import datetime

c:\Users\Noctris\.virtualenvs\rdf-literal-preprocessing-20b3_M0v\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dataload import amplus

In [32]:
data = amplus()

loaded data amplus (48.83s).
pruned (29.01s).


In [40]:
from preprocess import bin_numbers_10
data = bin_numbers_10(data)

10
deleting relations [826142, 826143], since no occurences are given


In [41]:
df =pd.DataFrame(data.triples, columns=["h","r","t"])
df['h'] = df['h'].apply(lambda h:data.i2e[h][0])
df['r'] = df['r'].apply(lambda r:data.i2r[r])
df['t'] = df['t'].apply(lambda t:data.i2e[t][0])

In [67]:
df[df['r']=='http://purl.org/collections/nl/am/dimensionValue']

,h,r,t
2,_:N00007dceb2094ae9823559779f4ea2eb,http://purl.org/collections/nl/am/dimensionValue,26
8,_:N00019150f29e40598930219eebc6761f,http://purl.org/collections/nl/am/dimensionValue,13.4
14,_:N0001b52ba5114ce6af33883e8f921899,http://purl.org/collections/nl/am/dimensionValue,6
18,_:N0001bb3b5430454eae404522eda3cb76,http://purl.org/collections/nl/am/dimensionValue,11.5
28,_:N00039ec0683b423a93f28f15afed6f81,http://purl.org/collections/nl/am/dimensionValue,27
...,...,...,...
846522,_:Nfffe9c2a85414ab7b065d38d6cca3364,http://purl.org/collections/nl/am/dimensionValue,4
846526,_:Nffff30c404e14aefae5c73a35a23d52c,http://purl.org/collections/nl/am/dimensionValue,46.8
846530,_:Nffff437703004c9483b446c765f45b35,http://purl.org/collections/nl/am/dimensionValue,908.2
846534,_:Nffff52dfd35e487da9a24ec608a07d38,http://purl.org/collections/nl/am/dimensionValue,0.3


In [66]:
('10', 'none') in data.i2e

True

In [68]:
df.to_csv(f'here.tsv.gz', 
        index=False,
        sep="\t" ,
        compression="gzip")

In [69]:
from pykeen.triples import TriplesFactory
b = TriplesFactory.from_path(f'here.tsv.gz')

In [72]:
for e in data.i2e:
    b.entity_to_id[e[0]]

In [61]:
df['r'].unique()

array(['http://purl.org/collections/nl/am/dimensionType',
       'http://purl.org/collections/nl/am/dimensionUnit',
       'http://purl.org/collections/nl/am/dimensionValue',
       'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
       'http://www.w3.org/1999/02/22-rdf-syntax-ns#value',
       'http://purl.org/collections/nl/am/exhibitionDateEnd',
       'http://purl.org/collections/nl/am/exhibitionDateStart',
       'http://purl.org/collections/nl/am/exhibitionLref',
       'http://purl.org/collections/nl/am/exhibitionOrganiser',
       'http://purl.org/collections/nl/am/exhibitionTitle',
       'http://purl.org/collections/nl/am/exhibitionVenue',
       'http://purl.org/collections/nl/am/biography',
       'http://purl.org/collections/nl/am/birthDateEnd',
       'http://purl.org/collections/nl/am/birthPlace',
       'http://purl.org/collections/nl/am/deathDateEnd',
       'http://purl.org/collections/nl/am/deathPlace',
       'http://purl.org/collections/nl/am/nationality',
     

In [65]:
df[df['r'] == 'http://master-thesis.com/predicat#binning11']['t'].unique()

array(['http://master-thesis.com/entity#binning1',
       'http://master-thesis.com/entity#binning2',
       'http://master-thesis.com/entity#binning10',
       'http://master-thesis.com/entity#binning5',
       'http://master-thesis.com/entity#binning3',
       'http://master-thesis.com/entity#binning6',
       'http://master-thesis.com/entity#binning4'], dtype=object)

In [46]:
from pykeen.triples import TriplesFactory
a = TriplesFactory.from_path(f'../data/preprocessed/amplus+bin_numbers_10-num_bins@10+delete_number_literals-neighborhood@1.tsv.gz')

In [56]:
a.entity_to_id['10']

KeyError: '10'

In [44]:
len(df[df['t'] == '10'])

888

In [27]:
df =pd.DataFrame(data.triples, columns=["h","r","t"])
df['r'] = df['r'].apply(lambda r:data.i2r[r][0])
df

,h,r,t
0,122633,h,812633
1,122633,h,826098
2,122633,h,13115
3,122633,h,508242
4,122636,h,508245
...,...,...,...
2346338,506593,h,826135
2346339,506620,h,826135
2346340,506664,h,826135
2346341,506695,h,826135


In [24]:
data.triples

tensor([[122633,      9, 812633],
        [122633,     10, 826098],
        [122633,     11,  13115],
        ...,
        [506664,     34, 826135],
        [506695,     34, 826135],
        [506746,     34, 826135]])

In [23]:
df

,h,r,t
0,_:N00007dceb2094ae9823559779f4ea2eb,h,http://purl.org/collections/nl/am/t-2763
1,_:N00007dceb2094ae9823559779f4ea2eb,h,0
2,_:N00007dceb2094ae9823559779f4ea2eb,h,26
3,_:N00007dceb2094ae9823559779f4ea2eb,h,http://purl.org/collections/nl/am/Dimension
4,_:N00016a5d0a164d9fb941dfa00b1bf444,h,http://purl.org/collections/nl/am/Maker
...,...,...,...
2346338,_:Nffe2e7f33e4e4774b9f69a7841a216f2,h,http://master-thesis.com/entity#binning1
2346339,_:Nffe6d4199bbf45ed8b2ccf879ccce2ab,h,http://master-thesis.com/entity#binning1
2346340,_:Nffedee7b3b0d421684cbb33fa4532c2f,h,http://master-thesis.com/entity#binning1
2346341,_:Nfff30055e2ea4b12b46986f13684bef3,h,http://master-thesis.com/entity#binning1


In [17]:
def encode_number_sublist(df: torch.Tensor, i2e: List[str]) -> torch.Tensor:
    sub_df = df.clone()
    for i in range(len(sub_df)):
        sub_df[i, 1] = torch.tensor(
            float(i2e[sub_df[i, 2]][0]), dtype=torch.float32)
    sub_df = sub_df[:, :2]
    return sub_df

def bin_numbers(data: Data, num_bins=3, use_lof=False, num_bins_as_percent=False, equal_height_binning=False, **kwargs):
    relevent_relations = get_relevant_relations(
        data, relevant_types=RDF_NUMBER_TYPES)
    print(num_bins)
    for b in range(num_bins):
        o = (f'{URI_PREFIX}entity#binning{b+1}', f'{URI_PREFIX}datatype#bin')
        new_id = len(data.i2e)
        data.e2i[o] = new_id
        data.i2e.append(o)
        data.num_entities += 1

    for r in relevent_relations:
        p = f'{URI_PREFIX}predicat#binning{r}'
        new_id = len(data.i2r)
        data.r2i[p] = new_id
        data.i2r.append(p)
        data.num_relations += 1

    for relation in relevent_relations:

        sub_df = encode_number_sublist(
            data.triples[data.triples[:, 1] == relation], data.i2e)

        # TODO test new function
        if (use_lof):
            lof = LocalOutlierFactor(n_neighbors=10)
            lof.fit(sub_df[:, 1].reshape(-1, 1))
            outlier_scores = lof.negative_outlier_factor_
            # Create a new column in the numpy array to store the outlier scores
            # tensor_np = torch.hstack((encoded_df, outlier_scores.reshape(-1,1)))
            threshold = np.percentile(outlier_scores, 10)
            # use the outlier scores to filter out the outliers from the numpy array
            sub_df = sub_df[outlier_scores > threshold]

        # numpy is used here since torch.histc was not working for some reason.
        sub_df = torch.cat(  # put bins and sub_df together
            (sub_df, torch.from_numpy(  # get numpy solutions back
                np.digitize(  # assign for each value in sub_df the corresponding bin
                    sub_df[:, 1], np.histogram(  # calculate n bins based on values in sub_df
                        sub_df[:, 1], num_bins)[1][:-1]
                )
            ).reshape(-1, 1)  # transfrom x tensor into (x,1) tensor to fit (x,2) shape of sub_df
            ), 1)

        object_mapping = np.vectorize(lambda t: data.e2i[(
            f'{URI_PREFIX}entity#binning{t}', f'{URI_PREFIX}datatype#bin')])

        predicat_mapping = np.vectorize(
            lambda t: data.r2i[f'{URI_PREFIX}predicat#binning{relation}'])

        sub_df[:, 1] = torch.tensor([predicat_mapping(sub_df[:, 2])], dtype=torch.int32)
        sub_df[:, 2] = torch.tensor([object_mapping(sub_df[:, 2])], dtype=torch.int32)
        data.triples = torch.cat((data.triples, sub_df), 0)
    #data = delete_empty_bin_types(data,num_bins)
    return data

def bin_numbers_10(data: Data, **kwargs):
    return bin_numbers(data=data, num_bins=10, use_lof=False)
    

In [59]:
data = bin_numbers_10(data)

10


In [19]:
len(torch.unique(data.triples[:,2]))

532380

In [61]:
def delete_empty_bin_types(data, num_bins):
    to_delete= []
    for d in data.i2e[-num_bins:]:
        filtered = data.triples[data.triples[:,2]==data.e2i[d]]
        if len(filtered) == 0:
            to_delete.append(data.e2i[d])
    if(len(to_delete)!=0):
        #to_delete_i = [data.e2i[d] in]
        print(f"deleting relations {to_delete}, since no occurences are given")
                #create new e mapping
        new_e2i = {}
        new_i2e = []

        for i in range(len(data.i2e)):
            if i not in to_delete:
                nt = torch.tensor(len(new_i2e), dtype=torch.int32)
                it =torch.tensor(i, dtype=torch.int32)
                new_e2i[data.i2e[it]] = nt
                new_i2e.append(data.i2e[it])
                # apply new mapping for triples
        for t in data.triples:
            t[0] = new_e2i[data.i2e[t[0]]]
            #t[1] = torch.tensor(data.r2i[data.i2r[t[1].numpy()]], dtype=torch.int32)
            t[2] = new_e2i[data.i2e[t[2]]]

            # create new train & withheld
        # new_train =  []
        # new_withheld = []

            # calculate new train & withheld
        #no need in thisuse case??
        #for ent in data.training:
        #    new_train.append([new_e2i[data.i2e[ent[0]]],ent[1]])
    
        #for ent in data.withheld:
        #    new_withheld.append([new_e2i[data.i2e[ent[0]]],ent[1]])

            #update metedata
        data.num_entities = len(new_i2e)


            #update data
            #data.triples = filtered
        data.i2e = new_i2e
        data.e2i = new_e2i

        #data.training = torch.tensor(new_train)
        #data.withheld = torch.tensor(new_withheld)
    return data    


In [58]:
data.i2e[data.triples[0,2]]

('http://purl.org/collections/nl/am/t-2763', 'iri')

In [32]:
from embed.pykeen_models import data_to_pykeen


In [28]:
data.name = "amplus+bin_numbers_10-num_bins@10+delete_number_literals-neighborhood@1"

In [34]:
data.name

'amplus+bin_numbers_10-num_bins@10+delete_number_literals-neighborhood@1'

In [35]:
from pykeen.triples import TriplesFactory
a =TriplesFactory.from_path(f'../data/preprocessed/amplus+bin_numbers_10-num_bins@10+delete_number_literals-neighborhood@1.tsv.gz')

In [48]:
len(data.e2i)

825677

In [47]:
len(a.entity_to_id)

817033

In [40]:
a.entity_to_id[' (1534-1603)']

2

In [42]:
data.i2e

[('', 'none'),
 ('                                                      sokkel in dep 3 kast 25',
  'none'),
 (' (1534-1603)', 'none'),
 (' (4 talen)', 'none'),
 (' (Duits)', 'none'),
 (' (Engels)', 'none'),
 (' (Frans)', 'none'),
 (' (Nederlands)', 'none'),
 (' (affiche)', 'none'),
 (' (verkeerspel)', 'none'),
 (' -1968)', 'none'),
 (' De Algemene Winkeliersvereniging aan mr. W.F. van Leeuwen bij zijn aftreden als burgemeester',
  'none'),
 (' Le canal du Heerengracht * bij de Spiegelstraat', 'none'),
 (' Natura Artis Magistra : 1838-1863 : Koninklijk Zoölogisch Genootschap',
  'none'),
 (' Transportnummer: 2009-0059                           in doos op grond',
  'none'),
 (' Transportnummer:2006-0316           doos 243', 'none'),
 (' We have no bananas', 'none'),
 (' brandewijnkom', 'none'),
 (' en scheepsmagazijn spuit 7 no. 27', 'none'),
 (' en scheepsmagazijn spuit R no. 35', 'none'),
 (' gegeven, tetjeroen zyn kluchtig leven. no. 27 (z.g. kinderprent)', 'none'),
 (" hierziet gij 

In [65]:
data.e2i[('10','none')]

tensor(507017, dtype=torch.int32)

In [66]:
len(data.i2e)

826143

In [64]:
data.i2e[data.e2i[('10','none')]]

('10', 'none')

In [57]:
type(data.e2i)

dict

In [67]:
from embed import TransE
model =TransE(data)

pykeen file does not exist. Writing pykeen file...


OSError: Cannot save file into a non-existent directory: 'data\preprocessed'

In [54]:

data.e2i

{('', 'none'): 0,
 ('                                                      sokkel in dep 3 kast 25',
  'none'): 1,
 (' (1534-1603)', 'none'): 2,
 (' (4 talen)', 'none'): 3,
 (' (Duits)', 'none'): 4,
 (' (Engels)', 'none'): 5,
 (' (Frans)', 'none'): 6,
 (' (Nederlands)', 'none'): 7,
 (' (affiche)', 'none'): 8,
 (' (verkeerspel)', 'none'): 9,
 (' -1968)', 'none'): 10,
 (' De Algemene Winkeliersvereniging aan mr. W.F. van Leeuwen bij zijn aftreden als burgemeester',
  'none'): 11,
 (' Le canal du Heerengracht * bij de Spiegelstraat', 'none'): 12,
 (' Natura Artis Magistra : 1838-1863 : Koninklijk Zoölogisch Genootschap',
  'none'): 13,
 (' Transportnummer: 2009-0059                           in doos op grond',
  'none'): 14,
 (' Transportnummer:2006-0316           doos 243', 'none'): 15,
 (' We have no bananas', 'none'): 16,
 (' brandewijnkom', 'none'): 17,
 (' en scheepsmagazijn spuit 7 no. 27', 'none'): 18,
 (' en scheepsmagazijn spuit R no. 35', 'none'): 19,
 (' gegeven, tetjeroen zyn 

In [43]:
reorder = []
for e in data.i2e:
    print(e)
    
    reorder.append(a.entity_to_id[e[0]])

('', 'none')
('                                                      sokkel in dep 3 kast 25', 'none')
(' (1534-1603)', 'none')
(' (4 talen)', 'none')
(' (Duits)', 'none')
(' (Engels)', 'none')
(' (Frans)', 'none')
(' (Nederlands)', 'none')
(' (affiche)', 'none')
(' (verkeerspel)', 'none')
(' -1968)', 'none')
(' De Algemene Winkeliersvereniging aan mr. W.F. van Leeuwen bij zijn aftreden als burgemeester', 'none')
(' Le canal du Heerengracht * bij de Spiegelstraat', 'none')
(' Natura Artis Magistra : 1838-1863 : Koninklijk Zoölogisch Genootschap', 'none')
(' Transportnummer: 2009-0059                           in doos op grond', 'none')
(' Transportnummer:2006-0316           doos 243', 'none')
(' We have no bananas', 'none')
(' brandewijnkom', 'none')
(' en scheepsmagazijn spuit 7 no. 27', 'none')
(' en scheepsmagazijn spuit R no. 35', 'none')
(' gegeven, tetjeroen zyn kluchtig leven. no. 27 (z.g. kinderprent)', 'none')
(" hierziet gij van Speyk's roemrijk leven, dade en dood. No 321 (z

KeyError: '10'

In [36]:
a.entity_to_id

{'': 0,
 '                                                      sokkel in dep 3 kast 25': 1,
 ' (1534-1603)': 2,
 ' (4 talen)': 3,
 ' (Duits)': 4,
 ' (Engels)': 5,
 ' (Frans)': 6,
 ' (Nederlands)': 7,
 ' (affiche)': 8,
 ' (verkeerspel)': 9,
 ' -1968)': 10,
 ' De Algemene Winkeliersvereniging aan mr. W.F. van Leeuwen bij zijn aftreden als burgemeester': 11,
 ' Le canal du Heerengracht * bij de Spiegelstraat': 12,
 ' Natura Artis Magistra : 1838-1863 : Koninklijk Zoölogisch Genootschap': 13,
 ' Transportnummer: 2009-0059                           in doos op grond': 14,
 ' Transportnummer:2006-0316           doos 243': 15,
 ' We have no bananas': 16,
 ' brandewijnkom': 17,
 ' en scheepsmagazijn spuit 7 no. 27': 18,
 ' en scheepsmagazijn spuit R no. 35': 19,
 ' gegeven, tetjeroen zyn kluchtig leven. no. 27 (z.g. kinderprent)': 20,
 " hierziet gij van Speyk's roemrijk leven, dade en dood. No 321 (z.g. kinderprent)": 21,
 ' in doos van ka 17208': 22,
 ' kom': 23,
 ' kommetje': 24,
 ' transpo

In [63]:
data.i2e[10]

(' -1968)', 'none')

In [62]:
data = delete_empty_bin_types(data,10)

deleting relations [826142, 826143], since no occurences are given


In [23]:
torch.unique(data.triples[:,2])

tensor([     0,      1,      2,  ..., 826140, 826141, 826142])

In [ ]:
# # get subset data
#     filtered = data.triples[~(torch.isin(data.triples[:,1],r))]
#     # get neg e filter
#     base_e_unique = torch.unique(torch.cat([data.triples[:,0],data.triples[:,2]])) 
#     filtered_e_unique = torch.unique(torch.cat([filtered[:,0],filtered[:,2]])) 
#     neg_e_filter =  base_e_unique[~(torch.isin(base_e_unique, filtered_e_unique))]

#     #get neg r filter
#     base_r_unique = torch.unique(data.triples[:,1])
#     filtered_r_unique = torch.unique(filtered[:,1])
#     neg_r_filter =  base_r_unique[~(torch.isin(base_r_unique, filtered_r_unique))]

#     #create new e mapping
#     new_e2i = {}
#     new_i2e = []

#     for i in range(len(data.i2e)):
#         if i not in neg_e_filter.numpy():
#             new_e2i[data.i2e[i]] = len(new_i2e)
#             new_i2e.append(data.i2e[i])

#     #create new r mapping
#     new_r2i = {}
#     new_i2r = []

#     for i in range(len(data.i2r)):
#         if i not in neg_r_filter.numpy():
#             new_r2i[data.i2r[i]] = len(new_i2r)
#             new_i2r.append(data.i2r[i])

#     # apply new mapping for triples
#     for t in filtered:
#         t[0] = torch.tensor(new_e2i[data.i2e[t[0].numpy()]], dtype=torch.int32)
#         t[1] = torch.tensor(new_r2i[data.i2r[t[1].numpy()]], dtype=torch.int32)
#         t[2] = torch.tensor(new_e2i[data.i2e[t[2].numpy()]], dtype=torch.int32)

#     # create new train & withheld
#     new_train =  []
#     new_withheld = []

#     # calculate new train & withheld
#     for ent in data.training:
#         new_train.append([new_e2i[data.i2e[ent[0].numpy()]],ent[1]])
    
#     for ent in data.withheld:
#         new_withheld.append([new_e2i[data.i2e[ent[0].numpy()]],ent[1]])

#     #update metedata
#     data.num_entities = len(new_i2e)
#     data.num_relations = len(new_i2r)

#     #update data
#     data.triples = filtered
#     data.i2e = new_i2e
#     data.e2i = new_e2i
#     data.i2r = new_i2r
#     data.r2i = new_r2i
#     data.training = torch.tensor(new_train)
#     data.withheld = torch.tensor(new_withheld)

In [20]:
data.triples[data.triples[:,2]== data.e2i[data.i2e[-3]]]

tensor([], size=(0, 3), dtype=torch.int64)

In [24]:
def delete_empty_bin_types(data, num_bins):
    for d in data.i2e[-num_bins:]:
        print(d)

In [40]:
delete_empty_bin_types(data,10)

In [21]:
data.i2e[-3]

('http://master-thesis.com/entity#binning8',
 'http://master-thesis.com/datatype#bin')

In [23]:
for d in data.i2e[-10:]:
    print(d)

('http://master-thesis.com/entity#binning1', 'http://master-thesis.com/datatype#bin')
('http://master-thesis.com/entity#binning2', 'http://master-thesis.com/datatype#bin')
('http://master-thesis.com/entity#binning3', 'http://master-thesis.com/datatype#bin')
('http://master-thesis.com/entity#binning4', 'http://master-thesis.com/datatype#bin')
('http://master-thesis.com/entity#binning5', 'http://master-thesis.com/datatype#bin')
('http://master-thesis.com/entity#binning6', 'http://master-thesis.com/datatype#bin')
('http://master-thesis.com/entity#binning7', 'http://master-thesis.com/datatype#bin')
('http://master-thesis.com/entity#binning8', 'http://master-thesis.com/datatype#bin')
('http://master-thesis.com/entity#binning9', 'http://master-thesis.com/datatype#bin')
('http://master-thesis.com/entity#binning10', 'http://master-thesis.com/datatype#bin')


In [12]:
data.e2i[data.i2e[-10]]

826135

In [7]:

print(num_bins)
for b in range(num_bins):
    o = (f'{URI_PREFIX}entity#binning{b+1}',f'{URI_PREFIX}datatype#bin')
    new_id = len(data.i2e)
    data.e2i[o] = new_id
    data.i2e.append(o)
    data.num_entities += 1
    if (verbose > 0):
        print(f'created new entity:')
        print(f'{data.e2i[o]} - {o}')



3
created new entity:
341271 - ('http://master-thesis.com/entity#binning1', 'http://master-thesis.com/datatype#bin')
created new entity:
341272 - ('http://master-thesis.com/entity#binning2', 'http://master-thesis.com/datatype#bin')
created new entity:
341273 - ('http://master-thesis.com/entity#binning3', 'http://master-thesis.com/datatype#bin')


In [8]:
print(relevent_relations)
for r in relevent_relations:
    p = f'{URI_PREFIX}predicat#binning{r}'
    new_id = len(data.i2r)
    data.r2i[p] = new_id
    data.i2r.append(p)
    data.num_relations += 1
    if (verbose > 0):
        print(f'created new relation:')
        print(f'{data.r2i[p]} - {p}')

[26, 2, 43]
created new relation:
60 - http://master-thesis.com/predicat#binning26
created new relation:
61 - http://master-thesis.com/predicat#binning2
created new relation:
62 - http://master-thesis.com/predicat#binning43


In [10]:
def encode_number_sublist(df: torch.Tensor, i2e: List[str]) -> torch.Tensor:
    sub_df = df.clone()
    for i in range(len(sub_df)):
        sub_df[i, 1] = torch.tensor(
            float(i2e[sub_df[i, 2]][0]), dtype=torch.float32)
    sub_df = sub_df[:, :2]
    return sub_df

In [29]:
print(len(data.triples))

777124


In [30]:
for rr in relevent_relations:
    print(rr)
    print(datetime.time())
    df = data.triples.clone()
    df = df[df[:, 1]== rr]
    sub_df = encode_number_sublist(df, data.i2e)
    print("sub2")
    sub_df = torch.cat( #put bins and sub_df together
        (sub_df, torch.from_numpy( #get numpy solutions back
            np.digitize( # assign for each value in sub_df the corresponding bin
                sub_df[:, 1], np.histogram( # calculate n bins based on values in sub_df
                    sub_df[:, 1], num_bins)[1][:-1]
                )
            ).reshape(-1, 1) # transfrom x tensor into (x,1) tensor to fit (x,2) shape of sub_df
        ), 1)
    
    ofn = lambda t: data.e2i[(f'{URI_PREFIX}entity#binning{t}',f'{URI_PREFIX}datatype#bin')]
    vofn = np.vectorize(ofn)
    pfn = lambda t: data.r2i[f'{URI_PREFIX}predicat#binning{rr}']
    vpfn = np.vectorize(pfn)
    to_add_df = sub_df.clone()
    print(len(to_add_df))
    #print(sub_df[:,2])
    print("aaa")
    to_add_df[:,1]= torch.tensor([vpfn(sub_df[:,2])], dtype=torch.int32)
    to_add_df[:,2]= torch.tensor([vofn(sub_df[:,2])],dtype=torch.int32)
    #print(to_add_df)
    #print(np.unique(vofn(sub_df[:,2])))
    #print(np.unique(vpfn(sub_df[:,2])))
    #print(vpfn(sub_df[:,2]))
    #print('eee')
    #print(num_bins)
    #print(torch.unique(sub_df[:,2]))
    #print("suib")
    data.triples = torch.cat((data.triples, to_add_df), 0)
    

26
00:00:00
sub2
510
aaa
2
00:00:00
sub2
8396
aaa
43
00:00:00
sub2
1800
aaa


In [31]:
print(len(data.triples))

787830


In [18]:
np.histogram( # calculate n bins based on values in sub_df
                    sub_df[:, 1], num_bins)[1][:-1]

array([4.50000e+01, 2.47242e+05, 4.94439e+05])

In [17]:
np.digitize( # assign for each value in sub_df the corresponding bin
                sub_df[:, 1], np.histogram( # calculate n bins based on values in sub_df
                    sub_df[:, 1], num_bins)[1][:-1]
                )

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
mport numpy as np
x = np.array([1, 2, 3, 4, 5])
squarer = lambda t: t ** 2
vfunc = np.vectorize(squarer)
vfunc(x)
# Output : array([ 1,  4,  9, 16, 25])

In [6]:
a = [2,5,4,6,2,6,7]
a[1:-1:2]

[5, 6, 6]

In [17]:
from pykeen.triples import TriplesFactory
import time
times= []
times.append(time.time())
print(times[0])
 
print('pykeen file does not exist. Writing pykeen file...')
# df = pd.DataFrame(columns=["h","r","t"])
        
# dfs = []
# for d in data.triples:
#     dfs.append(pd.DataFrame(
#         [[
#         data.i2e[d[0]][0],
#         data.i2r[d[1]],
#         data.i2e[d[2]][0]
#         ]],
#         columns=["h","r","t"]
#     ))
df =pd.DataFrame(data.triples, columns=["h","r","t"])
df['h'] = df['h'].apply(lambda h:data.i2e[h])
df['r'] = df['r'].apply(lambda r:data.i2r[r])
df['t'] = df['t'].apply(lambda t:data.i2e[t])
times.append(time.time())
print("iterated")
print(times[-1]-times[-2])


# # df = pd.concat(dfs,ignore_index=True)
# times.append(time.time())
# print("concat")
# print(times[-1]-times[-2])
df.to_csv(f'{data.name}.tsv.gz', 
    index=False,
    sep="\t" ,
    compression="gzip")
times.append(time.time())
print("to_csv")
print(times[-1]-times[-2])
a= TriplesFactory.from_path(f'{data.name}.tsv.gz')
times.append(time.time())
print("from_path")
print(times[-1]-times[-2])

1679934421.295006
pykeen file does not exist. Writing pykeen file...
iterated
1.4829964637756348
to_csv
45.34000325202942
from_path
17.014994859695435


In [13]:
dd =pd.DataFrame(data.triples, columns=["h","r","t"])

In [14]:

dd['h'] = dd['h'].apply(lambda h:data.i2e[h][0])
dd['r'] = dd['r'].apply(lambda r:data.i2r[r][0])
dd['t'] = dd['t'].apply(lambda t:data.i2e[t][0])

In [15]:
dd

,h,r,t
0,_:N00007dceb2094ae9823559779f4ea2eb,h,http://purl.org/collections/nl/am/t-2763
1,_:N00007dceb2094ae9823559779f4ea2eb,h,0
2,_:N00007dceb2094ae9823559779f4ea2eb,h,26
3,_:N00007dceb2094ae9823559779f4ea2eb,h,http://purl.org/collections/nl/am/Dimension
4,_:N00016a5d0a164d9fb941dfa00b1bf444,h,http://purl.org/collections/nl/am/Maker
...,...,...,...
2183987,http://purl.org/collections/nl/am/t-newgeuzen,h,http://www.w3.org/2004/02/skos/core#Concept
2183988,http://purl.org/collections/nl/am/t-newolympis...,h,http://www.w3.org/2004/02/skos/core#Concept
2183989,http://purl.org/collections/nl/am/t-newpolsband,h,http://www.w3.org/2004/02/skos/core#Concept
2183990,http://purl.org/collections/nl/am/t-newt-shirt,h,http://www.w3.org/2004/02/skos/core#Concept
